# Place a molecule onto a Surface

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int
from aiida.work.workfunction import workfunction

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview

from apps.surfaces.structure_browser import StructureBrowser

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        atoms = s.get_ase()
        nx, ny = guess_slab_size(atoms)
        nx_slider.value = nx
        ny_slider.value = ny
        orig_structure = s
    update_view()

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

## Step 2: Select cell size

In [ ]:
def on_nxyz_change(c):
    update_view()
    

drop_surface = ipw.Dropdown(description="Surface",
                             options=["Au(111)","Ag(111)","Cu(111)"]
#                           ,style=style, layout=layout
                            )
drop_surface.observe(on_nxyz_change, names='value')
nx_slider = ipw.IntSlider(description="nx", min=1, max=22, continuous_update=False)
nx_slider.observe(on_nxyz_change, names='value')
ny_slider = ipw.IntSlider(description="ny", min=1, max=16, continuous_update=False)
ny_slider.observe(on_nxyz_change, names='value')
nz_slider = ipw.IntSlider(description="nz", min=4, max=10, continuous_update=False)
nz_slider.observe(on_nxyz_change, names='value')

display(drop_surface,nx_slider, ny_slider, nz_slider)

In [ ]:
def update_view():

    with info_out:
        clear_output()

        inp_descr.value = ""

        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            nx = nx_slider.value
            ny = ny_slider.value
            nz = nz_slider.value
            which_surf=drop_surface.value
            inp_descr.value = orig_structure.description + "_" + which_surf + ".slab(%d,%d,%d)"%(nx,ny,nz)
            atoms = prepare_mol_on_slab(orig_structure.get_ase(), nx=nx, ny=ny, nz=nz,which_surf=which_surf)

            # add new component
            viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
            viewer.add_unitcell()
            viewer.center()

In [ ]:
viewer = nglview.NGLWidget()
info_out = ipw.Output()
display(viewer, info_out)

In [ ]:
# Au lattice param used: 4.1691
# dimensions of a rectangular unit cell of Au with z pointing in (111) direction
Au_x = 2.94799888 # a/sqrt(2)
Au_y = 5.10608384 # a*sqrt(3/2)

# Ag lattice param used: 4.10669535119
# dimensions of a rectangular unit cell of Ag with z pointing in (111) direction
Ag_x = 2.90387213 # a/sqrt(2)
Ag_y = 5.02965407 # a*sqrt(3/2)

# Cu lattice param used: 3.60103410694
# dimensions of a rectangular unit cell of Cu with z pointing in (111) direction
Cu_x = 2.54631564 # a/sqrt(2)
Cu_y = 4.41034805 # a*sqrt(3/2)

# And the x,y coordinates of the Au,Ag,Cu atoms in exact symmetric positions of the three layers of the unit cell
au_exact_xz = np.array([
    [[0.00000000, 1.70202795], [1.47399944, 4.25506987]],
    [[0.00000000, 0.00000000], [1.47399944, 2.55304192]],
    [[0.00000000, 3.40405590], [1.47399944, 0.85101397]]])

dz_au_bulk = 2.40703101 # Distance between bulk Au layers 
dz_au_h    = 0.86519004 # Distance between Au and H layer
dz_au_1_2  = 2.45156905 # Distance between Au top layer and 2nd layer
dz_au_2_3  = 2.39444938 # Distance between Au 2nd layer and 3rd layer (bulk)


# And the x,y coordinates of the Au,Ag,Cu atoms in exact symmetric positions of the three layers of the unit cell
ag_exact_xz = au_exact_xz*4.10669535119 / 4.1691

dz_ag_bulk = dz_au_bulk*4.10669535119 / 4.1691 # Distance between bulk Ag layers 
dz_ag_h    = 0.85106910 # Distance between Ag and H layer
dz_ag_1_2  = 2.35156179 # Distance between Ag top layer and 2nd layer
dz_ag_2_3  = 2.37580264 # Distance between Ag 2nd layer and 3rd layer (bulk)



# And the x,y coordinates of the Au,Ag,Cu atoms in exact symmetric positions of the three layers of the unit cell
cu_exact_xz = au_exact_xz*3.60103410694 / 4.1691

dz_cu_bulk = dz_au_bulk*3.60103410694 / 4.1691 # Distance between bulk Cu layers 
dz_cu_h    = 0.86502011 # Distance between Cu and H layer
dz_cu_1_2  = 2.06853511 # Distance between Cu top layer and 2nd layer
dz_cu_2_3  = 2.08397214 # Distance between Cu 2nd layer and 3rd layer (bulk)



def guess_slab_size(mol):
    cx = np.amax(mol.positions[:,0]) - np.amin(mol.positions[:,0]) + 10
    cy = np.amax(mol.positions[:,1]) - np.amin(mol.positions[:,1]) + 10
    nx = int(round(cx/Au_x))
    ny = int(round(cy/Au_y))
    return nx, ny

def prepare_mol_on_slab(mol, nx, ny, nz, which_surf):
    
    if which_surf == "Au(111)":
        Lx=Au_x
        Ly=Au_y
        dz_bulk=dz_au_bulk
        dz_h=dz_au_h
        dz_2_3=dz_au_2_3
        dz_1_2=dz_au_1_2
        exact_xz=au_exact_xz
        ida=79
    elif which_surf == "Ag(111)":
        Lx=Ag_x
        Ly=Ag_y
        dz_bulk=dz_ag_bulk
        dz_h=dz_ag_h
        dz_2_3=dz_ag_2_3
        dz_1_2=dz_ag_1_2
        exact_xz=ag_exact_xz
        ida=47
    elif which_surf == "Cu(111)":
        Lx=Cu_x
        Ly=Cu_y
        dz_bulk=dz_cu_bulk
        dz_h=dz_cu_h
        dz_2_3=dz_cu_2_3
        dz_1_2=dz_cu_1_2
        exact_xz=cu_exact_xz
        ida=29
    
    # Build up the unit cell of the gold slab with nz Au layers
    the_slab = []
    layer_z = 10.0
    # H layer in au lattice x-y positions
    the_slab.append([1, exact_xz[0, 0, 0], exact_xz[0, 0, 1], layer_z])
    the_slab.append([1, exact_xz[0, 1, 0], exact_xz[0, 1, 1], layer_z])
    # nz bulk layers
    for i in range(nz):
        if i == 0:
            layer_z += dz_h
        elif i == nz - 2:
            layer_z += dz_2_3
        elif i == nz - 1:
            layer_z += dz_1_2
        else:
            layer_z += dz_bulk
        xy_pos = exact_xz[(i+1)%3]
        the_slab.append([ida, xy_pos[0, 0], xy_pos[0, 1], layer_z])
        the_slab.append([ida, xy_pos[1, 0], xy_pos[1, 1], layer_z])
    
    the_slab = np.array(the_slab)
    
    # determine cell size
    vac_size = 40.0
    slab_z_max = np.max(the_slab[:, 3])
    slab_z_extent = slab_z_max - np.min(the_slab[:, 3])
    
    cz = slab_z_extent + vac_size
    cx = nx * Lx
    cy = ny * Ly
    print("Cell ABC: %f, %f, %f"%(cx, cy, cz))
    
    # position molecule
    mol_slab_dist = 2.3
    
    mol.cell = (cx,cy,cz)
    mol.pbc = (True,True,True)
    
    # position molecule a bit above gold slab
    mol.center()
    mol_z_min = np.min(mol.positions[:,2])
    mol.positions[:,2] += slab_z_max - mol_z_min + mol_slab_dist
    
    
    # generate gold slab based on the rectangular unit cell
    the_slab_raw = []
    for i in range(nx):
        for j in range(ny):
            shift = np.array([0, i*Lx, j*Ly, 0])
            the_slab_raw.append(the_slab + shift)

    the_slab_raw = np.concatenate(the_slab_raw)
    the_slab = ase.Atoms(numbers=the_slab_raw[:,0], positions=the_slab_raw[:,1:])
    the_slab = ase.build.sort(the_slab, tags=the_slab.get_positions()[:,2]*-1)
    
    mol_on_au = mol + the_slab
    return mol_on_au

## Step 3: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    if not orig_structure:
        print("No structure selected.")
        return

    nx = Int(nx_slider.value)
    ny = Int(ny_slider.value)
    nz = Int(nz_slider.value)
    s = prepare_mol_on_slab_wf(orig_structure, nx, ny, nz)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

cell_ready = False
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

In [ ]:
@workfunction
def prepare_mol_on_slab_wf(orig_struct, nx, ny, nz):
    nx, ny, nz = nx.value, ny.value, nz.value
    orig_atoms = orig_struct.get_ase()
    new_atoms = prepare_mol_on_slab(orig_atoms, nx, ny, nz, drop_surface.value)
    return StructureData(ase=new_atoms)